In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
file_name = 'proudboys.csv'
data = pd.read_csv(file_name, encoding='ISO-8859-1')
data = data[['user_id', 'status_id', 'text', 'Y/N']]
data['proc_text'] = data['text'].str.lower()
data.head(10)

C:\Users\Fahim\Anaconda3\envs\social_media_mining\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,status_id,text,Y/N,proc_text
0,x382691414,x1297365508300447744,Proud boys are attacking Peaceful protesters i...,N,proud boys are attacking peaceful protesters i...
1,x743936431,x1297365466306904066,"@Medde13 @MrAndyNgo I live in Portland, proud ...",N,"@medde13 @mrandyngo i live in portland, proud ..."
2,x743936431,x1294749099976036353,Anyone else wondering why tRump and his deplor...,"N, O",anyone else wondering why trump and his deplor...
3,x375304364,x1297365456190406662,Not Antifa but the Right Wing Proud Boys. If y...,Y,not antifa but the right wing proud boys. if y...
4,x176941218,x1297365411533643776,@MARCIAN2003 @JoshuaPotash The police and the ...,N,@marcian2003 @joshuapotash the police and the ...
5,x1119014143590305797,x1294746648653045760,"""Proud Boys"" are white supremacists, the Unite...","N, O","""proud boys"" are white supremacists, the unite..."
6,x1119014143590305797,x1297364973841154048,#TrumpGestapo allowing white supremacist DOMES...,"N, O",#trumpgestapo allowing white supremacist domes...
7,x1119014143590305797,x1297365393649139712,"#Trump's #VeryFinePeople, #DomesticTerrorists ...","N, O","#trump's #veryfinepeople, #domesticterrorists ..."
8,x724027390842703872,x1297365343690727424,"""The Proud Boys"" is a fitting name for the gro...","N, O","""the proud boys"" is a fitting name for the gro..."
9,x3327166976,x1297365325810262017,Proud Boys what the hell is a Proud Boy? Soun...,"N, O",proud boys what the hell is a proud boy? soun...


In [5]:
# Remove the unlabeled data
data = data.dropna(subset=['Y/N'])

In [6]:
data['supportive'] = data['Y/N'].str.contains('Y')
data['offensive'] = data['Y/N'].str.contains('O')
data.head(10)

,user_id,status_id,text,Y/N,proc_text,supportive,offensive
0,x382691414,x1297365508300447744,Proud boys are attacking Peaceful protesters i...,N,proud boys are attacking peaceful protesters i...,False,False
1,x743936431,x1297365466306904066,"@Medde13 @MrAndyNgo I live in Portland, proud ...",N,"@medde13 @mrandyngo i live in portland, proud ...",False,False
2,x743936431,x1294749099976036353,Anyone else wondering why tRump and his deplor...,"N, O",anyone else wondering why trump and his deplor...,False,True
3,x375304364,x1297365456190406662,Not Antifa but the Right Wing Proud Boys. If y...,Y,not antifa but the right wing proud boys. if y...,True,False
4,x176941218,x1297365411533643776,@MARCIAN2003 @JoshuaPotash The police and the ...,N,@marcian2003 @joshuapotash the police and the ...,False,False
5,x1119014143590305797,x1294746648653045760,"""Proud Boys"" are white supremacists, the Unite...","N, O","""proud boys"" are white supremacists, the unite...",False,True
6,x1119014143590305797,x1297364973841154048,#TrumpGestapo allowing white supremacist DOMES...,"N, O",#trumpgestapo allowing white supremacist domes...,False,True
7,x1119014143590305797,x1297365393649139712,"#Trump's #VeryFinePeople, #DomesticTerrorists ...","N, O","#trump's #veryfinepeople, #domesticterrorists ...",False,True
8,x724027390842703872,x1297365343690727424,"""The Proud Boys"" is a fitting name for the gro...","N, O","""the proud boys"" is a fitting name for the gro...",False,True
9,x3327166976,x1297365325810262017,Proud Boys what the hell is a Proud Boy? Soun...,"N, O",proud boys what the hell is a proud boy? soun...,False,True


In [7]:
stemmer = PorterStemmer()

for idx, row in data.iterrows():
    words = nltk.word_tokenize(row['proc_text'])
    words = [stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
    words = [word for word in words if word.isalpha()]
    data.loc[idx, 'proc_text'] = ' '.join(words)

data.head(10)

,user_id,status_id,text,Y/N,proc_text,supportive,offensive
0,x382691414,x1297365508300447744,Proud boys are attacking Peaceful protesters i...,N,proud boy attack peac protest portland portlan...,False,False
1,x743936431,x1297365466306904066,"@Medde13 @MrAndyNgo I live in Portland, proud ...",N,mrandyngo live portland proud boy white suprem...,False,False
2,x743936431,x1294749099976036353,Anyone else wondering why tRump and his deplor...,"N, O",anyon els wonder trump deplor base speak conde...,False,True
3,x375304364,x1297365456190406662,Not Antifa but the Right Wing Proud Boys. If y...,Y,antifa right wing proud boy think far left cau...,True,False
4,x176941218,x1297365411533643776,@MARCIAN2003 @JoshuaPotash The police and the ...,N,joshuapotash polic proud boy,False,False
5,x1119014143590305797,x1294746648653045760,"""Proud Boys"" are white supremacists, the Unite...","N, O",proud boy white supremacist unit state declar ...,False,True
6,x1119014143590305797,x1297364973841154048,#TrumpGestapo allowing white supremacist DOMES...,"N, O",trumpgestapo allow white supremacist domest te...,False,True
7,x1119014143590305797,x1297365393649139712,"#Trump's #VeryFinePeople, #DomesticTerrorists ...","N, O",trump veryfinepeopl domesticterrorist proud bo...,False,True
8,x724027390842703872,x1297365343690727424,"""The Proud Boys"" is a fitting name for the gro...","N, O",proud boy fit name group consid none emot poli...,False,True
9,x3327166976,x1297365325810262017,Proud Boys what the hell is a Proud Boy? Soun...,"N, O",proud boy hell proud boy sound like puss fill ...,False,True


In [9]:
np.sum(data['supportive'])

757

In [8]:
data.shape

(2014, 7)

In [10]:
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000)
X1 = cv.fit_transform(data['proc_text']).toarray()

In [11]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=2000)
X2 = cv.fit_transform(data['proc_text']).toarray()

In [25]:
# Labels for Supportive / Non supportive
y = pd.get_dummies(data['supportive'])
y = y.iloc[:, 1].values
print(y[:6])

[0 0 0 1 0 0]


In [36]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.20, random_state=0)

In [37]:
from sklearn.naive_bayes import MultinomialNB
model_NB = MultinomialNB().fit(X_train, y_train)

y_pred = model_NB.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)

In [39]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7468982630272953
